In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress



In [2]:
#Filtering down columns
#Disaster Decloration Summaries - disasterNumber, state, declarationType, fyDeclared, incidentType,  
#Hazard Mitigation Assistance Projects - disasterNumber, projectType, numberOfProperties, numberOfFinalProperties, projectAmount, federalShareObligated, benefitCostRatio, netValueBenefits
#Hazard Mitigation Grant Program Disaster Summaries - disasterNumber and obligatedTotalAmount

In [3]:
#Disaster Decloration Summaries - disasterNumber, state, declarationType, fyDeclared, incidentType,  
disaster_sum = pd.read_csv('DisasterDeclarationsSummaries.csv')
disaster_sum_clean = disaster_sum[['disasterNumber', 'state', 'declarationType', 'fyDeclared', 'incidentType']]
disaster_sum_clean.head()

,disasterNumber,state,declarationType,fyDeclared,incidentType
0,1,GA,DR,1953,Tornado
1,4,MI,DR,1953,Tornado
2,3,LA,DR,1953,Flood
3,6,MI,DR,1953,Tornado
4,2,TX,DR,1953,Tornado


In [4]:
#Hazard Mitigation Assistance Projects - disasterNumber, projectType, numberOfProperties, numberOfFinalProperties, projectAmount, federalShareObligated, benefitCostRatio, netValueBenefits
haz_project = pd.read_csv('HazardMitigationAssistanceProjects.csv')
haz_project_clean = haz_project[['disasterNumber', 'projectType', 'numberOfProperties', 'numberOfFinalProperties', 'projectAmount', 'federalShareObligated', 'benefitCostRatio', 'netValueBenefits', 'projectIdentifier']]
haz_project_clean.head()

,disasterNumber,projectType,numberOfProperties,numberOfFinalProperties,projectAmount,federalShareObligated,benefitCostRatio,netValueBenefits,projectIdentifier
0,1971.0,206.1: Safe Room (Tornado and Severe Wind Shel...,5,4,31477.0,20086.0,1.371,21420.0,DR-1971-0561-R
1,1971.0,206.1: Safe Room (Tornado and Severe Wind Shel...,4,4,19032.0,13650.0,1.371,21420.0,DR-1971-0428-R
2,1971.0,206.2: Safe Room (Tornado and Severe Wind Shel...,1,0,769334.0,577000.0,4.460,3181553.0,DR-1971-0430-R
3,1971.0,601.1: Generators,0,0,42578.0,31934.0,0.000,0.0,DR-1971-0373-F
4,1971.0,206.2: Safe Room (Tornado and Severe Wind Shel...,1,1,143899.0,107924.0,3.097,500916.0,DR-1971-0672-R


In [23]:
disaster_projects = pd.merge(haz_project_clean, disaster_sum_clean, on='disasterNumber')
disaster_summary = disaster_projects.groupby(['state','incidentType', 'disasterNumber', 'projectIdentifier']).mean()
disaster_summary.head(40)

numberOfProperties  \
state incidentType  disasterNumber projectIdentifier                       
AK    Coastal Storm 4244.0         DR-4244-0001-P                      0   
                                   DR-4244-0002-P                      0   
                                   DR-4244-0003-R                      0   
                                   DR-4244-0004-F                      0   
                                   DR-4244-0007-M                      0   
                    4369.0         DR-4369-0003-M                      0   
      Earthquake    1440.0         DR-1440-0001-M                      0   
                                   DR-1440-0002-R                      1   
                                   DR-1440-0004-P                      0   
                                   DR-1440-0005-P                      0   
                                   DR-1440-0006-P                      0   
                                   DR-1440-0007-P                      0   
                                   DR-1440-0008-R                     16   
                                   DR-1440-0009-R                      0   
                                   DR-1440-0010-P                      0   
                                   DR-1440-0012-R                      0   
                                   DR-1440-0013-R                      0   
                                   DR-1440-0014-P                      0   
                                   DR-1440-0015-R                      0   
                    4413.0         DR-4413-0001-M                      0   
      Fire          1119.0         DR-1119-0002-F                      0   
                                   DR-1119-0003-F                      0   
                                   DR-1119-0004-F                      0   
                                   DR-1119-0005-R                      0   
                                   DR-1119-0006-R                      0   
                                   DR-1119-0007-R                      0   
                                   DR-1119-0008-R                      0   
                                   DR-1119-0009-R                      0   
                                   DR-1119-0010-R                      0   
                                   DR-1119-0011-R                      0   
                                   DR-1119-0012-R                      1   
                                   DR-1119-0013-R                      1   
                                   DR-1119-0014-R                      0   
                                   DR-1119-0015-F                      0   
                                   DR-1119-M001-M                      0   
                    1666.0         DR-1666-0001-R                      1   
                                   DR-1666-0002-R                      1   
                                   DR-1666-0003-R                      0   
                                   DR-1666-0004-R                      0   
                                   DR-1666-0005-M                      0   

                                                      numberOfFinalProperties  \
state incidentType  disasterNumber projectIdentifier                            
AK    Coastal Storm 4244.0         DR-4244-0001-P                           0   
                                   DR-4244-0002-P                           0   
                                   DR-4244-0003-R                           0   
                                   DR-4244-0004-F                           0   
                                   DR-4244-0007-M                           0   
                    4369.0         DR-4369-0003-M                           0   
      Earthquake    1440.0         DR-1440-0001-M                           0   
                                   DR-1440-0002-R                           1   
                                   DR-1440-0004-P  

In [24]:
len(disaster_summary)

22179

In [10]:
#Hazard Mitigation Grant Program Disaster Summaries
hazard_sum = pd.read_csv('HazardMitigationGrantProgramDisasterSummaries.csv')
hazard_sum_clean = hazard_sum[['disasterNumber', 'obligatedTotalAmount']]
hazard_sum_clean.head()

,disasterNumber,obligatedTotalAmount
0,1071,2834575.0
1,1337,782081.0
2,1326,333150.0
3,1506,1791043.0
4,1211,413384.0
